Chúng ta sẽ kết hợp các model với nhau theo technical indicator:
1. Model rf
2. Model xgb
3. Model LSTM
4. Model MLP
5. Model naive bayes
6. Logistic regression

In [ ]:
!pip install pandas_ta

In [2]:
# pip install pandas_ta --upgrade



In [3]:
import numpy as np
import pandas as pd
import pickle
import joblib
import zipfile
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import xgboost as xgb
import math
import pandas_ta as ta
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
%matplotlib inline

In [4]:
drive.mount("/content/drive", force_remount = True)

MessageError: ignored

In [ ]:
file_path_forex = "/content/drive/MyDrive/forex_data/forex_data/forex_data.pkl"

In [ ]:
with open(file_path_forex, "rb") as forex:
  data = pickle.load(forex)

In [ ]:
keys_list = list(data.keys())
print(keys_list)
data_all_money = [pd.DataFrame(k) for k in data.values()]

In [ ]:
import pandas as pd
import pandas_ta as ta

def calculate_technical_indicators(data):
    df = pd.DataFrame(data)


    df.ta.sma(length=14, append=True)  # Simple Moving Average (Điểm trung bình đơn giản)
    df.ta.ema(length=14, append=True)  # Exponential Moving Average (Điểm trung bình mở rộng)
    df.ta.macd(append=True)  # Moving Average Convergence Divergence (Động hóa hợp đồng trung bình)
    df.ta.rsi(length=14, append=True)  # Relative Strength Index (Chỉ số sức mạnh tương đối)
    df.ta.bbands(length=20, append=True)  # Bollinger Bands (Dải Bollinger)
    df.ta.adx(append=True)  # Average Directional Index (Chỉ số hướng trung bình)
    df.ta.stoch(append=True)  # Stochastic Oscillator (Dao động Stochastic)
    df.ta.cci(append=True)  # Commodity Channel Index (Chỉ số kênh hàng hoá)
    df.ta.atr(append=True)  # Average True Range (Phạm vi biến động trung bình)
    df.ta.vortex(append=True)  # Vortex Indicator (Chỉ số xoáy)
    df.ta.kdj(append=True)  # Stochastic Oscillator KDJ (Dao động Stochastic KDJ)
    df.ta.mfi(append=True)  # Money Flow Index (Chỉ số dòng tiền)
    df.ta.roc(append=True)  # Rate of Change (Tỷ lệ thay đổi)
    df.ta.ppo(append=True)  # Percentage Price Oscillator (Dao động giá phần trăm)
    df.ta.trix(append=True)  # Triple Exponential Average (Điểm trung bình ba lần)
    df.ta.massi(append=True)  # Mass Index (Chỉ số Mass)
    df.ta.willr(append=True)  # Williams %R (Chỉ số Williams %R)
    df.ta.mom(append=True)  # Momentum (Động lượng)

    df.ta.cmo(append=True)  # Chande Momentum Oscillator (Chande Momentum Oscillator)
    df.ta.ppo(append=True)  # Percentage Price Oscillator (Dao động giá phần trăm)




In [ ]:
# for money in data_all_money:
#   calculate_technical_indicators(money)

In [ ]:
# data_all_money[0].columns

In [ ]:
def edit_trend(data):
    data['Trend'] = 0  # Tạo cột Trend và gán giá trị mặc định là 0
    for i in range(len(data) - 1):
        if data['Close'][i] < data['Close'][i + 1]:
            data['Trend'][i] = 1
        else:
            data['Trend'][i] = -1

    # Gán giá trị mặc định cho ngày cuối cùng
    data['Trend'].iloc[-1] = 1

In [ ]:
for money in data_all_money:
  edit_trend(money)

In [ ]:
def tinh_loi_lo_thang(df, month, year):
    filtered_data = df[(df.index.month == month) & (df.index.year == year)].copy()
    profit = 0.0

    for i in range(len(filtered_data)):
        # Lấy giá close hiện tại và giá close tiếp theo
        current_close = filtered_data.iloc[i]['Close']
        next_close = filtered_data.iloc[i+1]['Close'] if i+1 < len(filtered_data) else None
        prediction = filtered_data.iloc[i]['predict']

        # Tính lợi/lỗ
        if prediction == 1:
            if next_close is not None:
                profit += next_close - current_close
        else:
            if next_close is not None:
                profit += current_close - next_close

    return profit

    # [20, 10, -1, 5]
    # [1, -1, 1]
    # 10-20 =-10, 10 - (-1) =11, 5 -(-1) = 6
    # 6+ (-10)+11 =7

In [ ]:
def fill_missing_values(data, fill_with=0):
    # Kiểm tra và thay thế NaN bằng giá trị fill_with
    data.fillna(fill_with, inplace=True)

    # Kiểm tra và thay thế các cột không có giá trị (tất cả giá trị là NaN) bằng giá trị fill_with
    for column in data.columns:
        if data[column].isnull().all():
            data[column] = fill_with


In [ ]:
def create_features_time(df, label=None):
    df['date'] = df.index
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear

    df.drop(columns=["date"], inplace=True)  # Chú ý tới drop(columns=...) và inplace=True

for type_money in data_all_money:
  create_features_time(type_money)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

def calculate_metrics(y_test, y_pred):
    # Tính Accuracy, Precision và Recall sử dụng scikit-learn
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    return accuracy, precision, recall



In [ ]:
split_date = '01-Jun-2021'
data_EURCHF_copy = data_all_money[0].copy()
fill_missing_values(data_EURCHF_copy)
data_train_EURCHF_copy = data_EURCHF_copy.loc[data_EURCHF_copy.index <= split_date].copy()
data_test_EURCHF_copy = data_EURCHF_copy.loc[data_EURCHF_copy.index > split_date].copy()

y_train_EURCHF_copy = data_train_EURCHF_copy["Trend"]
y_test_EURCHF_copy = data_test_EURCHF_copy["Trend"]
data_train_EURCHF_copy.drop(["Trend"], axis = 1, inplace =True)
data_test_EURCHF_copy.drop(["Trend"], axis = 1, inplace =True)

y_train_EURCHF_copy_01 =  [1 if label == 1 else 0 for label in y_train_EURCHF_copy]
y_train_EURCHF_copy_01 = pd.DataFrame({'y_train_EURCHF_copy_01': y_train_EURCHF_copy_01})

y_test_EURCHF_copy_01 =  [1 if label == 1 else 0 for label in y_test_EURCHF_copy]
y_test_EURCHF_copy_01 = pd.DataFrame({'y_test_EURCHF_copy_01': y_test_EURCHF_copy_01})


In [ ]:
# def create_rf_model(x_train, y_train):
#   rf =  RandomForestClassifier(n_estimators = 400, max_depth = 10, random_state = 14,
#                             min_samples_split= 5,min_samples_leaf = 3, max_features= 2)
#   rf.fit(x_train, y_train)
#   return rf

In [ ]:
# model_rf = create_rf_model(data_train_EURCHF_copy, y_train_EURCHF_copy_01)

In [ ]:
# def create_xgb_model(x_train, y_train):
#     xgb_model = xgb.XGBClassifier(
#         n_estimators=400,
#         max_depth=10,
#         learning_rate=0.1,
#         random_state=14,
#         min_child_weight=1,
#         gamma=0,
#         subsample=0.8,
#         colsample_bytree=0.8,
#         reg_alpha=0.005,
#         objective='binary:logistic',
#         scale_pos_weight=1,  # Chỉ định scale_pos_weight ở đây cho 1 và -1
#         seed=27
#     )

#     xgb_model.fit(x_train, y_train)
#     return xgb_model

In [ ]:
# model_xgb = create_xgb_model(data_train_EURCHF_copy, y_train_EURCHF_copy_01)

model lstm

In [ ]:
# from tensorflow.keras import layers
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.model_selection import train_test_split

# # Tạo một lớp tối ưu hóa với learning rate tùy chỉnh
#   # Thay đổi learning rate tại đây

# def create_LSTM_model(x_train, y_train, random_state=42):
#     x_train_arr = x_train.values
#     y_train_arr = y_train.values
#     x_train_arr = x_train_arr.reshape(x_train_arr.shape[0], x_train_arr.shape[1], 1)
#     y_train_arr = y_train_arr.reshape(-1, 1)
#     custom_optimizer = keras.optimizers.Adam(learning_rate=0.001)

#     # Phân chia dữ liệu thành tập huấn luyện và tập kiểm tra với random_state
#     x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_arr, y_train_arr, test_size=0.2, random_state=random_state)

#     # Tạo mô hình Sequential
#     model = keras.Sequential()

#     # Thêm lớp LSTM với 50 đơn vị và đầu vào có shape (số quan sát, số đặc trưng, chiều dài chuỗi)
#     model.add(layers.LSTM(216, input_shape=(x_train_arr.shape[1], x_train_arr.shape[2]), activation='relu', return_sequences=True))
#     model.add(layers.LSTM(128, activation='relu'))

#     # Thêm lớp BatchNormalization để chuẩn hóa đầu ra từ lớp LSTM
#     model.add(layers.BatchNormalization())

#     # Thêm lớp Dropout để tránh overfitting
#     model.add(layers.Dropout(0.1))

#     # Thêm 4 lớp Dense với activation là 'tanh' trong khoảng từ -1 đến 1
#     model.add(layers.Dense(64, activation=lambda x: tf.keras.activations.relu(x) * 2))
#     model.add(layers.Dense(32, activation='relu'))

#     # Thêm lớp Dense với 1 đơn vị (dự đoán nhị phân)
#     model.add(layers.Dense(16, activation='sigmoid'))
#     model.add(layers.Dense(1, activation='sigmoid'))

#     # Biên soạn mô hình với tối ưu hóa tùy chỉnh
#     model.compile(loss='mse', optimizer=custom_optimizer, metrics=['accuracy'])

#     # Định nghĩa callback EarlyStopping
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#     # Huấn luyện mô hình với dữ liệu đào tạo và callback EarlyStopping
#     model.fit(x_train_split, y_train_split, epochs=10, batch_size=30, callbacks=[early_stopping], validation_data=(x_val_split, y_val_split))
#     return model

In [ ]:
model_lstm = create_LSTM_model(data_train_EURCHF_copy, y_train_EURCHF_copy_01)

model MLP

In [ ]:
# model_MLP = MLPClassifier(
#     hidden_layer_sizes=(100, 50),  # Cấu hình 2 lớp ẩn
#     activation='relu',             # Sử dụng hàm kích hoạt ReLU
#     solver='adam',                 # Sử dụng thuật toán tối ưu hóa Adam
#     alpha=0.0001,                  # Tham số kiểm soát tỷ lệ lỗi chuẩn
#     batch_size=14,                 # Kích thước batch
#     learning_rate_init=0.001,      # Tốc độ học ban đầu
#     max_iter=300,                 # Số lượng vòng lặp tối đa
#     random_state=14                # Giá trị random state
# )

# model_MLP.fit(data_train_EURCHF_copy, y_train_EURCHF_copy_01)







In [ ]:
# y_pred_MLP = model_MLP.predict(data_test_EURCHF_copy)
# y_pred_MLP

In [ ]:
# acc, pre, rec = calculate_metrics(y_test_EURCHF_copy_01, y_pred_MLP)
# print(acc)
# print(pre)
# print(rec)

In [ ]:

# from sklearn.naive_bayes import GaussianNB
# model_naive_bayes = GaussianNB()
# model_naive_bayes.fit(data_train_EURCHF_copy, y_train_EURCHF_copy_01)


In [ ]:
# y_pre_bayes = model_naive_bayes.predict(data_test_EURCHF_copy)

In [ ]:
# y_pre_bayes

In [ ]:
# acc, prec, recall = calculate_metrics(y_test_EURCHF_copy_01, y_pre_bayes)
# print(acc)
# print(prec)
# print(recall)

In [ ]:
data_EURCHF_copy.head(

)

In [ ]:
# rf_model = create_rf_model(data_train_EURCHF_copy, y_train_EURCHF_copy)

In [ ]:
# joblib.dump(rf_model, 'random_forest_model_technical_indicator.pkl')

In [ ]:
# xgb_model = create_xgb_model(data_train_EURCHF_copy, y_train_EURCHF_copy)

In [ ]:
# joblib.dump(xgb_model, 'xgb_model_technical_indicator.pkl')

In [ ]:
# from tensorflow.keras.models import load_model

# Tải lại mô hình từ tệp đã lưu
# model_lstm.save('lstm_model_technical_indicator.h5')


In [ ]:
# x_test_arr_lstm = data_test_EURCHF_copy.values.reshape(data_test_EURCHF_copy.shape[0],
#                                                 data_test_EURCHF_copy.shape[1], 1)

# y_pred_lstm  =  lstm_model.predict(x_test_arr_lstm)

In [ ]:
def compare_negative_and_positive(element):
      # So sánh và trả về kết quả
    if element >= 0:
        return 1
    else:
        return -1

def compare_negative_and_positive_array(input_array):
    # Sử dụng list comprehension để áp dụng hàm compare_negative_and_positive cho từng phần tử trong mảng đầu vào
    result_array = [compare_negative_and_positive(element) for element in input_array]

    return result_array


In [ ]:
# acc, prec, recall = calculate_metrics(y_test_EURCHF_copy, y_pred_lstm)
# print(acc)
# print(prec)
# print(recall)

Load toàn bộ model để stacking

In [ ]:
# def stack_model(model_rf, model_xgb, model_lstm, model_mlp, model_naive, x_test):
#     # Dự đoán kết quả từ các mô hình
#     y_hat_rf = model_rf.predict(x_test)
#     y_hat_xgb = model_xgb.predict(x_test)

#     x_test_rf = x_test.values.reshape(x_test.shape[0], x_test.shape[1], 1)
#     y_hat_lstm = model_lstm.predict(x_test_rf)
#     y_hat_lstm = compare_negative_and_positive_array(y_hat_lstm)

#     y_hat_mlp = model_mlp.predict(x_test)
#     y_hat_bayes = model_naive.predict(x_test)

#     # Tạo một danh sách (list) chứa các dự đoán từ các mô hình
#     predictions = [y_hat_rf, y_hat_xgb, y_hat_lstm, y_hat_mlp, y_hat_bayes]

#     # Tạo một mảng (array) chứa max_vote của từng mẫu dự đoán
#     max_vote_array = []
#     for i in range(len(x_test)):
#         total_votes = sum([1 if pred[i] == 1 else 0 for pred in predictions])
#         max_vote = 1 if total_votes >= 3 else 0
#         max_vote_array.append(max_vote)

#     return max_vote_array


In [ ]:
# y_hat_stack = stack_model(model_rf, model_xgb, model_lstm, model_MLP, model_naive_bayes, data_test_EURCHF_copy)

In [ ]:
# acc, prec, rec = calculate_metrics(y_test_EURCHF_copy_01, y_hat_stack)
# print(acc)
# print(prec)
# print(rec)

In [ ]:
from collections import Counter

def find_faulty_dates(y_test, y_pred):
    # Tạo danh sách các vị trí sai lệch
    faulty_indices = [i for i, (true, pred) in enumerate(zip(y_test, y_pred)) if true != pred]

    # Tạo danh sách các ngày tương ứng với các vị trí sai lệch
    faulty_dates = data_test_EURCHF_copy.index[faulty_indices]
    return faulty_dates

y_hat_stack_ = [1 if ele == 1 else -1 for ele in y_hat_stack]

# Sử dụng hàm để tìm kiếm các ngày bị lỗi
faulty_dates = find_faulty_dates(y_test_EURCHF_copy, y_hat_stack_)
print(faulty_dates)
# Tạo danh sách các tháng và năm lỗi
faulty_months_years = [(date.month, date.year) for date in faulty_dates]

# Đếm số lần xuất hiện của mỗi tháng và năm lỗi
month_year_counts = Counter(faulty_months_years)

# Sắp xếp theo chiều giảm dần của count
sorted_month_year_counts = sorted(month_year_counts.items(), key=lambda x: x[1], reverse=True)

# In ra các tháng và năm lỗi theo chiều giảm dần của count
print()
for (month, year), count in sorted_month_year_counts:
    print()
    print(f"Tháng {month}/{year} có tổng cộng {count} lần dự đoán sai")


In [ ]:
data_test_EURCHF_copy_money = data_test_EURCHF_copy.copy()
data_test_EURCHF_copy_money["predict"] = y_hat_stack

In [ ]:
a = tinh_loi_lo_thang(data_test_EURCHF_copy_money,month=2, year = 2022 )
print(a)

In [ ]:
print(tinh_loi_lo_thang(data_test_EURCHF_copy_money,month=9, year = 2021 ))
print(tinh_loi_lo_thang(data_test_EURCHF_copy_money,month=10, year = 2022 ))


In [ ]:
def view_EMA_plot_data(start, end): # start and end type is string
  # Đặt thời gian bắt đầu và kết thúc theo yêu cầu của bạn
  start_date = pd.Timestamp(start)
  end_date = pd.Timestamp(end)

  # Lọc dữ liệu theo khoảng thời gian
  filtered_data = data_test_EURCHF_copy_money[(data_test_EURCHF_copy_money.index >= start_date) & (data_test_EURCHF_copy_money.index <= end_date)]
  # Tìm giá trị tối đa và tối thiểu của dữ liệu trong khoảng thời gian
  max_value = filtered_data["EMA_14"].max()
  min_value = filtered_data["EMA_14"].min()
  plt.figure(figsize=(15, 6))

  # Vẽ đường biểu đồ EMA
  plt.plot(filtered_data.index, filtered_data["EMA_14"], label="EMA", color="blue")

  # Đặt tiêu đề cho biểu đồ
  plt.title("Biểu đồ EMA từ {} đến {}".format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))

  # Hiển thị hình chú thích (legend) cho đường EMA
  plt.legend(loc="upper left")

  # Hiển thị lưới (grid) cho trục y
  plt.grid(True)

  # Đặt giới hạn cho trục y theo giá trị tối đa và tối thiểu của dữ liệu trong khoảng thời gian
  plt.ylim(min_value, max_value)

  # Hiển thị biểu đồ
  plt.show()

start = "2021-9-1"
end = "2021-9-30"
view_EMA_plot_data(start, end)

In [ ]:

start = "2022-9-1"
end = "2022-9-30"
view_EMA_plot_data(start, end)

In [ ]:
start_date = "2022-9-1"
end_date = "2022-9-30"
filtered_data = data_test_EURCHF_copy_money[(data_test_EURCHF_copy_money.index >= start_date) & (data_test_EURCHF_copy_money.index <= end_date)]
trend_fil = y_test_EURCHF_copy[(y_test_EURCHF_copy.index >= start_date) & (y_test_EURCHF_copy.index <= end_date)]
filtered_data = filtered_data[["Close", "predict"]]
filtered_data["Trend"] = trend_fil


In [ ]:
def sap_xep_tuan_loi_nhieu_nhat(df, month, year, n):
    filtered_data = df[(df.index.month == month) & (df.index.year == year)].copy()
    week_profits = []

    # Lấy số lượng tuần trong tháng
    total_weeks = len(filtered_data) // n

    for i in range(total_weeks):
        # Lấy giá close đầu và cuối của tuần
        start_index = i * n
        end_index = min((i + 1) * n, len(filtered_data))
        current_close = filtered_data.iloc[start_index]['Close']
        end_close = filtered_data.iloc[end_index - 1]['Close']
        prediction = filtered_data.iloc[start_index]['predict']

        # Tính lợi/lỗ
        if prediction == 1:
            profit = end_close - current_close
        else:
            profit = current_close - end_close

        week_profits.append(profit)

    # Sắp xếp tuần theo lợi nhuận giảm dần và lấy chỉ số của tuần
    sorted_weeks = sorted(range(total_weeks), key=lambda i: week_profits[i], reverse=True)

    # Tạo danh sách tuần theo thứ tự lợi nhuận giảm dần (index + 1)
    sorted_weeks = [x + 1 for x in sorted_weeks]

    return sorted_weeks


In [ ]:
sap_xep_tuan_loi_nhieu_nhat(data_test_EURCHF_copy_money,month=9, year = 2021 , n=4)

In [ ]:
sap_xep_tuan_loi_nhieu_nhat(data_test_EURCHF_copy_money,month=9, year = 2022 , n=4)


In [ ]:
filtered_data

In [ ]:
start_date = "2021-9-1"
end_date = "2021-9-30"
filtered_data = data_test_EURCHF_copy_money[(data_test_EURCHF_copy_money.index >= start_date) & (data_test_EURCHF_copy_money.index <= end_date)]
trend_fil = y_test_EURCHF_copy[(y_test_EURCHF_copy.index >= start_date) & (y_test_EURCHF_copy.index <= end_date)]
filtered_data = filtered_data[["Close", "predict"]]
filtered_data["Trend"] = trend_fil
filtered_data